## **Sabrina Zheng | Group 6 | Project 1**

### 

_Written in collaboration with ChatGPT from OpenAI to improve understanding, assist with the explanation of the query, and enhance formatting and display of the queries._

### **Complex Queries:**

### **Query 1**

**Proposition:** <span style="color: var(--vscode-foreground);">Identify the most frequently occuring Pokemon for both the primary type and secondary type categories accross the three Pokemon generations and ranks the counts based on it and select the type with the highest occurance in each category.</span>

**Tables:**

- PokemonGen1.dbo.PokemonGen1
- PokemonGen2.dbo.PokemonGen2
- PokemonGen3.dbo.PokemonGen3

**Columns:**

- Type1: The primary type of a Pokemon.
- Type2: The secondary type of a Pokemon, which can be NULL if the Pokemon does not possess a secondary type.

**Predicate:**

- The query first aggregates the counts of each Pokemon for both Type1 and Type2 across all three generations in two separate CTE tables, Type1Counts and Type2Counts. It counts the number of occurances in the tables, excluding the null values. 
- Then it ranks the types based on the numbner of occurances using the RANK() function with two CTEs, RankType1 and RankType2
- The query then selects the most common types from RankedType1 and RankedType2 where the rank is 1 by using a SELECT statement with a UNION ALL
- If there are duplicates, or a case where the count is the same, all of them are diplayed

In [18]:
WITH Type1Counts AS (
    SELECT Type1 AS Type, COUNT(*) AS Count
    FROM PokemonGen1.dbo.PokemonGen1
    WHERE Type1 IS NOT NULL
    GROUP BY Type1
    UNION ALL
    SELECT Type1, COUNT(*)
    FROM PokemonGen2.dbo.PokemonGen2
    WHERE Type1 IS NOT NULL
    GROUP BY Type1
    UNION ALL
    SELECT Type1, COUNT(*)
    FROM PokemonGen3.dbo.PokemonGen3
    WHERE Type1 IS NOT NULL
    GROUP BY Type1
),
Type2Counts AS (
    SELECT Type2 AS Type, COUNT(*) AS Count
    FROM PokemonGen1.dbo.PokemonGen1
    WHERE Type2 IS NOT NULL
    GROUP BY Type2
    UNION ALL
    SELECT Type2, COUNT(*)
    FROM PokemonGen2.dbo.PokemonGen2
    WHERE Type2 IS NOT NULL
    GROUP BY Type2
    UNION ALL
    SELECT Type2, COUNT(*)
    FROM PokemonGen3.dbo.PokemonGen3
    WHERE Type2 IS NOT NULL
    GROUP BY Type2
),
RankedType1 AS (
    SELECT Type, Count, RANK() OVER (ORDER BY Count DESC) AS Rank
    FROM Type1Counts
),
RankedType2 AS (
    SELECT Type, Count, RANK() OVER (ORDER BY Count DESC) AS Rank
    FROM Type2Counts
)

SELECT 'Type1' AS Category, Type, Count
FROM RankedType1
WHERE Rank = 1
UNION ALL
SELECT 'Type2' AS Category, Type, Count
FROM RankedType2
WHERE Rank = 1;


(4 rows affected)

Total execution time: 00:00:00.003

Category,Type,Count
Type1,Water,28
Type2,Flying,19
Type2,Poison,19
Type2,Flying,19


### **Query 2**

**Proposition:** Write a query that identifies the Pokemon types that were first introduced in Generation 2 and continue to appear in Generation 3 that illustrates the evolution and retention of Pokemon types across these generations.

**Tables:**

- PokemonGen1.dbo.PokemonGen1
- PokemonGen2.dbo.PokemonGen2
- PokemonGen3.dbo.PokemonGen3

**Columns:**

- Type1: The primary type of Pokémon.
- Type2: The secondary type of Pokémon, which can be NULL if the Pokemon does not have a secondary type.

**Predicate:**

- It uses CTEs named Gen1Types, Gen2Types, and Gen3Types and gets a distinct list of both Type1 and Type2, exlcuding null
- The CTE then fillters the types from Gen2Types by exclluding the ones in Gen1Types
- The CTE continues the filter the types from Gen3Types by excluding the ones in Gen2Types
- FInally the SELECT statement is used to list all the types that are introduced in Gen2 and are continued in Gen3. The output shows the types that were introduced in the middle generation and then adopted in the next generation.

In [21]:
WITH Gen1Types AS (
    SELECT DISTINCT Type1 AS Type FROM PokemonGen1.dbo.PokemonGen1
    UNION
    SELECT DISTINCT Type2 FROM PokemonGen1.dbo.PokemonGen1 WHERE Type2 IS NOT NULL
), 
Gen2Types AS (
    SELECT DISTINCT Type1 AS Type FROM PokemonGen2.dbo.PokemonGen2
    UNION
    SELECT DISTINCT Type2 FROM PokemonGen2.dbo.PokemonGen2 WHERE Type2 IS NOT NULL
),
Gen3Types AS (
    SELECT DISTINCT Type1 AS Type FROM PokemonGen3.dbo.PokemonGen3
    UNION
    SELECT DISTINCT Type2 FROM PokemonGen3.dbo.PokemonGen3 WHERE Type2 IS NOT NULL
),
IntroducedInGen2 AS (
    SELECT Type FROM Gen2Types
    WHERE Type NOT IN (SELECT Type FROM Gen1Types)
),
ContinuedInGen3 AS (
    SELECT Type FROM IntroducedInGen2
    WHERE Type IN (SELECT Type FROM Gen3Types)
)

SELECT Type
FROM ContinuedInGen3;


(1 row affected)

Total execution time: 00:00:00.013

Type
Dark


### **Query 3**

**Proposition:** Write a query to find the Pokemon that has the highest attack in Type1 and Type2 within each generation

**Tables:**

- PokemonGen1.dbo.PokemonGen1
- PokemonGen2.dbo.PokemonGen2
- PokemonGen3.dbo.PokemonGen3

**Columns:**

- ID (or PokemonNoGen1, PokemonNoGen2, PokemonNoGen3): The unique identifier for a Pokemon in each generation.
- Name: The name of the Pokemon.
- Attack: The Attack statistic of the Pokemon.
- Type1 and Type2: Represent the primary and secondary types of the Pokemon.
- Generation: A derived column indicating the Pokemon's generation.

**Predicate:**

- The UnifiedPokemon CTE is used to create a unified view of the Pokemon from all 3 generations by getting the IDs, names, attack values, types, and assigns a generation tag based on the source table
- Then the TypeRankings CTE creates a temporary dataset that consolidates the Pokemon types and then ranks the Pokemon by their attack values within each type and generation using the RANK() function
- The query then selects the TypeRankings CTE and filters for when the Pokemon with the highest attack within the type and generation

In [30]:
WITH UnifiedPokemon AS (
    SELECT PokemonNoGen1 AS ID, Name, Attack, Type1, Type2, 'Gen1' AS Generation FROM PokemonGen1.dbo.PokemonGen1
    UNION ALL
    SELECT PokemonNoGen2 AS ID, Name, Attack, Type1, Type2, 'Gen2' FROM PokemonGen2.dbo.PokemonGen2
    UNION ALL
    SELECT PokemonNoGen3 AS ID, Name, Attack, Type1, Type2, 'Gen3' FROM PokemonGen3.dbo.PokemonGen3
),
TypeRankings AS (
    SELECT
        Name,
        Attack,
        Type,
        Generation,
        RANK() OVER(PARTITION BY Type, Generation ORDER BY Attack DESC) AS RankInType
    FROM (
        SELECT Name, Attack, Type1 AS Type, Generation FROM UnifiedPokemon
        UNION ALL
        SELECT Name, Attack, Type2, Generation FROM UnifiedPokemon WHERE Type2 IS NOT NULL
    ) AS AllTypes
)

SELECT Type, Generation, Name, Attack
FROM TypeRankings
WHERE RankInType = 1
ORDER BY Type, Generation;


(62 rows affected)

Total execution time: 00:00:00.010

Type,Generation,Name,Attack
Bug,Gen1,Pinsir,125
Bug,Gen2,Scizor,130
Bug,Gen3,Armaldo,125
Dark,Gen2,Tyranitar,134
Dark,Gen3,Absol,130
Dragon,Gen1,Dragonite,134
Dragon,Gen2,Kingdra,95
Dragon,Gen3,Rayquaza,150
Electric,Gen1,Zapdos,90
Electric,Gen1,Raichu,90


### **Query 4**

**Proposition:** Create a query that finds the most common abilities for Ability1, Ability2, and Ability3 from the 3 Pokemon generations

**Tables:**

- PokemonGen1.dbo.PokemonGen1: Contains data about Pokemon from the first generation, including their abilities.
- PokemonGen2.dbo.PokemonGen2: Contains data about Pokemon from the second generation, including their abilities.
- PokemonGen3.dbo.PokemonGen3: Contains data about Pokemon from the third generation, including their abilities.

**Columns:**

- AbilitySlot: Indicates which slot the ability belongs to (Ability1, Ability2, or Ability3).
- Ability: Represents the specific ability of a Pokemon.
- Count: Indicates how many times each ability occurs within its respective slot.

**Predicate:**

- Using the Abilities CTE, take all the abilites from the Pokemon from 3 generations and assign each ability to a slot in the table unless it is null
- Use the AbilityCounts CTE to group the AbilitySlot and Ability that keeps track of the count in each slot
- Then the MaxCounts CTE is used to count the abilites in each slot and determines the max count in each group
- If there are duplicates, they are still shown

In [34]:
WITH Abilities AS (
    SELECT 'Ability1' AS AbilitySlot, Ability1 AS Ability FROM PokemonGen1.dbo.PokemonGen1
    UNION ALL
    SELECT 'Ability2' AS AbilitySlot, Ability2 FROM PokemonGen1.dbo.PokemonGen1 WHERE Ability2 IS NOT NULL
    UNION ALL
    SELECT 'Ability3' AS AbilitySlot, Ability3 FROM PokemonGen1.dbo.PokemonGen1 WHERE Ability3 IS NOT NULL
    UNION ALL
    SELECT 'Ability1' AS AbilitySlot, Ability1 FROM PokemonGen2.dbo.PokemonGen2
    UNION ALL
    SELECT 'Ability2' AS AbilitySlot, Ability2 FROM PokemonGen2.dbo.PokemonGen2 WHERE Ability2 IS NOT NULL
    UNION ALL
    SELECT 'Ability3' AS AbilitySlot, Ability3 FROM PokemonGen2.dbo.PokemonGen2 WHERE Ability3 IS NOT NULL
    UNION ALL
    SELECT 'Ability1' AS AbilitySlot, Ability1 FROM PokemonGen3.dbo.PokemonGen3
    UNION ALL
    SELECT 'Ability2' AS AbilitySlot, Ability2 FROM PokemonGen3.dbo.PokemonGen3 WHERE Ability2 IS NOT NULL
    UNION ALL
    SELECT 'Ability3' AS AbilitySlot, Ability3 FROM PokemonGen3.dbo.PokemonGen3 WHERE Ability3 IS NOT NULL
),
AbilityCounts AS (
    SELECT AbilitySlot, Ability, COUNT(*) AS Count
    FROM Abilities
    GROUP BY AbilitySlot, Ability
),
MaxCounts AS (
    SELECT AbilitySlot, MAX(Count) AS MaxCount
    FROM AbilityCounts
    GROUP BY AbilitySlot
)

SELECT ac.AbilitySlot, ac.Ability, ac.Count
FROM AbilityCounts ac
JOIN MaxCounts mc ON ac.AbilitySlot = mc.AbilitySlot AND ac.Count = mc.MaxCount
ORDER BY ac.AbilitySlot, ac.Count DESC;


(6 rows affected)

Total execution time: 00:00:00.005

AbilitySlot,Ability,Count
Ability1,Chlorophyll,19
Ability1,Swift Swim,19
Ability2,Early Bird,9
Ability2,Flash Fire,9
Ability2,Sturdy,9
Ability3,Sheer Force,10


### **Query 5**

**Proposition:** Write a query that retrieves the Pokemon from all 3 generations that have a Defense value higher than the Special Defense Value

**Tables:**

- PokemonGen1.dbo.PokemonGen1: Contains data about Pokemon from the first generation, including their names, Defense, and SpecialDefense stats.
- PokemonGen2.dbo.PokemonGen2: Contains data about Pokemon from the second generation, including their names, Defense, and SpecialDefense stats.
- PokemonGen3.dbo.PokemonGen3: Contains data about Pokemon from the third generation, including their names, Defense, and SpecialDefense stats.

**Columns:**

- Generation: Indicates the generation of each Pokemon.
- Name: Represents the name of each Pokemon.
- Defense: Shows the Defense stat value of each Pokemon.
- SpecialDefense: Represents the Special Defense stat value of each Pokemon.

**Predicate:**

- There are 3 subqueries for each generation of Pokemon, it takes the generation name and then retrieves the name of the Pokemon, Defense, and SpecialDefense stats from the Pokemon table, then the data is filtered to only include the Pokemon that have a Defense higher than the Special Defense
- The UNION ALL operator is then used to combine the results from all three of the subqueries
- The outer query then takes the columns from the combined results and adds a Generation column to specify the generation of the Pokemon

In [35]:
SELECT *
FROM (
    SELECT 'Generation 1' AS Generation, Name, Defense, SpecialDefense
    FROM PokemonGen1.dbo.PokemonGen1
    WHERE Defense > SpecialDefense

    UNION ALL

    SELECT 'Generation 2' AS Generation, Name, Defense, SpecialDefense
    FROM PokemonGen2.dbo.PokemonGen2
    WHERE Defense > SpecialDefense

    UNION ALL

    SELECT 'Generation 3' AS Generation, Name, Defense, SpecialDefense
    FROM PokemonGen3.dbo.PokemonGen3
    WHERE Defense > SpecialDefense
) AS AllGenerations
ORDER BY Generation, Name;


(123 rows affected)

Total execution time: 00:00:00.008

Generation,Name,Defense,SpecialDefense
Generation 1,Rhydon,120,45
Generation 1,Tangela,115,40
Generation 1,Bellsprout,35,30
Generation 1,Caterpie,35,20
Generation 1,Cloyster,180,45
Generation 1,Cubone,95,50
Generation 1,Dodrio,70,60
Generation 1,Doduo,45,35
Generation 1,Exeggcute,80,45
Generation 1,Exeggutor,85,75


### **Query 6**

**Proposition:** Write a query that calculates the average values of the attributes of Pokemon: HP, Attack, Defense, Special Attack, Special Defense, and Speed across all 3 generations

**Tables:**

- PokemonGen1.dbo.PokemonGen1: Contains data about Pokemon from the first generation, including their attributes such as HP, Attack, Defense, Special Attack, Special Defense, and Speed.
- PokemonGen2.dbo.PokemonGen2: Contains data about Pokemon from the second generation, including their attributes.
- PokemonGen3.dbo.PokemonGen3: Contains data about Pokemon from the third generation, including their attributes.

**Columns:**

- HP: Represents the Hit Points attribute of Pokemon, indicating their health.
- Attack: Denotes the Attack attribute of Pokemon, representing their physical offensive power.
- Defense: Indicates the Defense attribute of Pokemon, showing their physical defensive capability.
- SpecialAttack: Represents the Special Attack attribute of Pokemon, indicating their offensive power for special moves.
- SpecialDefense: Denotes the Special Defense attribute of Pokemon, representing their defensive capability against special moves.
- Speed: Represents the Speed attribute of Pokemon, showing how fast they can act in battles.

**Predicate:**

- The UNION ALL is used with a subquery that takes the data from all 3 generations of Pokemon to combine the  HP, Attack, Defense, Special Attack, Special Defense, and Speed
- The outer query then calculates the average of each attribute using the AVG() function

In [38]:
SELECT 
    AVG(HP) AS Average_HP,
    AVG(Attack) AS Average_Attack,
    AVG(Defense) AS Average_Defense,
    AVG(SpecialAttack) AS Average_SpecialAttack,
    AVG(SpecialDefense) AS Average_SpecialDefense,
    AVG(Speed) AS Average_Speed
FROM (
    SELECT HP, Attack, Defense, SpecialAttack, SpecialDefense, Speed
    FROM PokemonGen1.dbo.PokemonGen1

    UNION ALL

    SELECT HP, Attack, Defense, SpecialAttack, SpecialDefense, Speed
    FROM PokemonGen2.dbo.PokemonGen2

    UNION ALL

    SELECT HP, Attack, Defense, SpecialAttack, SpecialDefense, Speed
    FROM PokemonGen3.dbo.PokemonGen3
) AS AllGenerations;

(1 row affected)

Total execution time: 00:00:00.003

Average_HP,Average_Attack,Average_Defense,Average_SpecialAttack,Average_SpecialDefense,Average_Speed
66,71,68,66,67,64


### **Query 7**

**Proposition:** Write a query that compares the average stats for HP, Attack, Defense, Special Attack, Special Defense, and Speed for all 3 generations of Pokemon

**Tables:**

- PokemonGen1.dbo.PokemonGen1: Contains data about Pokemon from the first generation, including their various attributes such as HP, Attack, Defense, Special Attack, Special Defense, and Speed.
- PokemonGen2.dbo.PokemonGen2: Contains data about Pokemon from the second generation, including their attributes.
- PokemonGen3.dbo.PokemonGen3: Contains data about Pokemon from the third generation, including their attributes.

**Columns:**

- Generation: Represents the generation of Pokemon for which the average statistics are calculated.
- AvgHP: Represents the average Hit Points (HP) across all Pokemon in the specified generation.
- AvgAttack: Represents the average Attack stat across all Pokemon.
- AvgDefense: Represents the average Defense stat across all Pokemon.
- AvgSpecialAttack: Represents the average Special Attack stat across all Pokemon.
- AvgSpecialDefense: Represents the average Special Defense stat across all Pokemon.
- AvgSpeed: Represents the average Speed stat across all Pokemon.

**Predicate:**

- The ConbinedStats CTE is used to calculate the average for all the attributes that the Pokemon have
- Then there are 3 subqueries that are used for each generation to calculate the average
- The UNION ALL operator is then used to combine the results of the subqueries in to the CombinedStats table

In [40]:
WITH CombinedStats AS (
    SELECT 'Gen1' AS Generation, 
           AVG(HP) AS AvgHP, 
           AVG(Attack) AS AvgAttack, 
           AVG(Defense) AS AvgDefense,
           AVG(SpecialAttack) AS AvgSpecialAttack,
           AVG(SpecialDefense) AS AvgSpecialDefense,
           AVG(Speed) AS AvgSpeed
    FROM PokemonGen1.dbo.PokemonGen1
    UNION ALL
    SELECT 'Gen2', 
           AVG(HP), 
           AVG(Attack), 
           AVG(Defense),
           AVG(SpecialAttack),
           AVG(SpecialDefense),
           AVG(Speed)
    FROM PokemonGen2.dbo.PokemonGen2
    UNION ALL
    SELECT 'Gen3', 
           AVG(HP), 
           AVG(Attack), 
           AVG(Defense),
           AVG(SpecialAttack),
           AVG(SpecialDefense),
           AVG(Speed)
    FROM PokemonGen3.dbo.PokemonGen3
)
SELECT Generation, 
       AvgHP, 
       AvgAttack, 
       AvgDefense,
       AvgSpecialAttack,
       AvgSpecialDefense,
       AvgSpeed
FROM CombinedStats;


(3 rows affected)

Total execution time: 00:00:00.006

Generation,AvgHP,AvgAttack,AvgDefense,AvgSpecialAttack,AvgSpecialDefense,AvgSpeed
Gen1,63,72,68,67,65,69
Gen2,70,68,69,64,72,61
Gen3,65,73,69,67,66,61
